In [ ]:
import os
os.chdir('./..')

import csv
import pickle
import Methods.totals as totals
import Methods.analysis as analysis
import Methods.common as common

with open("sorted_all_dataset.pkl", "rb") as f:
    dataset_dict = pickle.load(f)

In [ ]:
"""
    Getting totals
    Getting the final classification for every PR and calculating totals
"""
repos = 342
for repo_file in range(0, repos):
    variant1 = dataset_dict[repo_file]['source']
    variant2 = dataset_dict[repo_file]['destination']
    
    print(variant1, '-', variant2)
    
    try:
        all_results = common.readResults(repo_file, variant1)
        
        pr_class = totals.final_class(all_results[0])
        all_counts = totals.count_all_classifications(pr_class)

        with open('Repos_totals/'+ str(repo_file) + '_' + variant1.split('/')[0] + '_' + variant1.split('/')[1] + '_totals.pkl', 'wb') as f:
            pickle.dump([pr_class, all_counts],f)
    except Exception as e:
        print(e)

In [ ]:
"""
    Plotting the distribution of classifications
"""

total_all = 0
total_mo_all = 0
total_ed_all = 0
total_sp_all = 0
total_na_all = 0

all_results = []

repos = 342
for repo_file in range(0,repos):
    try:
        variant1 = dataset_dict[repo_file]['source']
        variant2 = dataset_dict[repo_file]['destination']
        variant1_prs = dataset_dict[repo_file]['pr'].split('/')

        all_totals= totals.read_totals(repo_file, variant1)

        total_NA = 0
        total_ED = 0
        total_MO = 0
        total_CC = 0
        total_SP= 0
        total_NE = 0
        total_ERROR = 0

        for pr in all_totals[0]:
            verdict = all_totals[0][pr]['class']
            if verdict == 'ED':
                total_ED += 1
            elif verdict =='MO':
                total_MO += 1
            elif verdict == 'SP':
                total_SP += 1
            elif verdict == 'NA':
                total_NA += 1
            elif verdict == 'CC':
                total_CC += 1
            elif verdict =='NE':
                total_NE += 1
            elif verdict == 'ERROR':
                total_ERROR += 1
                
            total_mid = total_MO+ total_ED + total_SP
            total_all += total_mid
            
        total_total =len(variant1_prs)
        
        total_mo_all += total_MO
        total_ed_all += total_ED
        total_sp_all += total_SP
        total_na_all += total_NA

        totals_list = [total_MO, total_ED, total_SP, total_CC, total_NE, total_NA, total_ERROR]

        analysis.all_class_bar(totals_list, repo_file, variant1, variant2, False)
#         analysis.all_class_pie(totals_list, repo_file, variant1, variant2, False)

        all_results.append([variant1, variant2, repo_file, total_MO, total_ED, total_SP, total_CC, total_NE, total_NA, total_ERROR])

    except Exception as e:
        print(variant1)
        print("Exception: ", e, '\n\n')

header = ["variant1", "variant2", "repo_file", "total_MO", "total_ED", "total_SP", "total_CC", "total_NE", "total_NA"," total_ERROR"]
with open("results.csv", "w") as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(all_results)

In [ ]:
print('MO = ',total_mo_all)
print('ED = ',total_ed_all)
print('SP = ',total_sp_all)